<a href="https://www.kaggle.com/code/potalatarun/fire-smoke-logistic?scriptVersionId=204676598" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/fire-smoke-and-neutral/FIRE-SMOKE-DATASET/Test'):
    # print(filenames)
    for drn,_,fln in os.walk(dirname):
        print(fln)
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[]
['image_22.jpg', 'image_53.jpg', 'image_69.jpg', 'image_68.jpg', 'image_51.jpg', 'image_88.jpg', 'image_4.jpg', 'image_75.jpg', 'image_12.jpg', 'image_16.jpg', 'image_72.jpg', 'image_58.jpg', 'image_62.jpg', 'image_2.jpg', 'image_45.jpg', 'image_7.jpg', 'image_85.jpg', 'image_65.jpg', 'image_5.jpg', 'image_86.jpg', 'image_99.jpg', 'image_36.jpg', 'image_18.jpg', 'image_33.jpg', 'image_19.jpg', 'image_31.jpg', 'image_27.jpg', 'image_66.jpg', 'image_61.jpg', 'image_92.jpg', 'image_11.jpg', 'image_39.jpg', 'image_74.jpg', 'image_79.jpg', 'image_26.jpg', 'image_28.jpg', 'image_44.jpg', 'image_14.jpg', 'image_48.jpg', 'image_41.jpg', 'image_10.jpg', 'image_81.jpg', 'image_47.jpg', 'image_30.jpg', 'image_77.jpg', 'image_59.jpg', 'image_23.jpg', 'image_60.jpg', 'image_63.jpg', 'image_34.jpg', 'image_94.jpg', 'image_50.jpg', 'image_32.jpg', 'image_70.jpg', 'image_83.jpg', 'image_98.jpg', 'image_21.jpg', 'image_95.jpg', 'image_93.jpg', 'image_25.jpg', 'image_0.jpg', 'image_17.jpg', 'image_87

1. Load the Pre-trained CNN:
    
    1.1. Import a CNN (e.g., VGG16, ResNet50) from a deep learning library like TensorFlow or PyTorch.

    1.2. Remove the top classification layer of the network to get the feature map output instead. For instance, VGG16 outputs a 4096-dimensional feature vector from its fully connected layers.

In [2]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
import numpy as np

# Load pre-trained VGG16 model and exclude the final classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


2. Extract Features:

    2.1 Preprocess each image (resize, normalize) to fit the input requirements of the CNN model.
   
    2.2 Pass each image through the feature extractor and flatten the output to create a vector of features.

In [3]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

def preprocess_and_extract(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    # Extract features
    features = feature_extractor.predict(img_array)
    features = features.flatten()
    return features


3: Train the Logistic Regression Model

   3.1 Create a Dataset of Extracted Features:

        
        For each image, extract its feature vector and pair it with the class label (e.g., 1 for fire/smoke and 0 for none).

In [4]:
# this cell here we are extracting the img_paths that it nothing do with the above markdown cell
import os
img_paths = []
labels = []
paths = ['Fire','Smoke','Neutral']

for path in paths:
    PATH = "/kaggle/input/fire-smoke-and-neutral/FIRE-SMOKE-DATASET/Train/"
    PATH = PATH+path
    for dirname, _, filenames in os.walk(f'{PATH}'):
        print(dirname)
        print(len(filenames))
        for filename in filenames:
            img_path = os.path.join(dirname,filename)
            img_paths.append(img_path)
            if path=='Fire':
                labels.append(1)
            elif path=="Smoke":
                labels.append(2)
            else:
                labels.append(0)
            # print(os.path.join(dirname, filename))
print(len(img_paths))
print(len(labels))

/kaggle/input/fire-smoke-and-neutral/FIRE-SMOKE-DATASET/Train/Fire
900
/kaggle/input/fire-smoke-and-neutral/FIRE-SMOKE-DATASET/Train/Smoke
900
/kaggle/input/fire-smoke-and-neutral/FIRE-SMOKE-DATASET/Train/Neutral
900
2700
2700


In [5]:
# this cell for the annotations
# this cell is for creating or taking label from the annotaions folder and this cell nothing to do with the above markdown cell this is just the helper cell
# import xml.etree.ElementTree as ET
# labels = []

# for dirname,_,filenames in os.walk('/kaggle/input/fire-and-smoke-dataset/Annotations/Annotations'):
#     for filename in filenames:
#         # Load the XML file
#         img_path = os.path.join(dirname,filename)
#         tree = ET.parse(img_path)
#         root = tree.getroot()
#         # Assuming each label is under a tag (like 'object' or 'label')
#         for elem in root.findall('.//object'):  # change 'object' based on your XML structure
#             label = elem.find('name').text  # 'name' is usually the tag for the label, adjust if different
#             labels.append(label)
# # Display the labels
# print(labels)

In [6]:
# Assume img_paths contains paths to images and labels has corresponding labels (1 or 0)
feature_vectors = [preprocess_and_extract(img_path) for img_path in img_paths]
feature_vectors = np.array(feature_vectors)
labels = np.array(labels)

I0000 00:00:1730467233.056946      64 service.cc:145] XLA service 0x7bdb64004880 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730467233.057038      64 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


I0000 00:00:1730467234.978825      64 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━

3.2 Train the Logistic Regression Model:

    With the extracted feature vectors, train a logistic regression model on this new dataset.

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the data
X_train, X_test, y_train, y_test = train_test_split(feature_vectors, labels, test_size=0.2, random_state=42)

# Initialize and train logistic regression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Evaluate
y_pred = log_reg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.95


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Step 4: Make Predictions

With the trained logistic regression model, you can now make predictions on new images. Just preprocess and extract features as before, then pass these features to the logistic regression model.

In [8]:
def predict_fire_smoke(img_path):
    features = preprocess_and_extract(img_path)
    prediction = log_reg.predict([features])
    # return "Fire/Smoke Detected" if prediction[0] == 1 else "No Fire/Smoke Detected"
    if prediction == 1:
        return "Fire"
    elif prediction == 2:
        return "Smoke"
    elif prediction == 0:
        return "Neutral"

# Example usage
print(predict_fire_smoke("/kaggle/input/fire-smoke-and-neutral/FIRE-SMOKE-DATASET/Test/Fire/image_0.jpg"))
print(predict_fire_smoke("/kaggle/input/fire-smoke-and-neutral/FIRE-SMOKE-DATASET/Test/Neutral/image_0.jpg"))
print(predict_fire_smoke("/kaggle/input/fire-smoke-and-neutral/FIRE-SMOKE-DATASET/Test/Smoke/image_0.jpg"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Fire
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Neutral
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Smoke


##### why did i try the above method
 -->Summary -->

    The pre-trained CNN extracts meaningful features from each image.

    The logistic regression model, trained on these features, acts as a classifier to detect fire or smoke.
        
    This approach leverages the powerful feature extraction capabilities of CNNs while keeping the classification model lightweight with logistic regression.